In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np


In [ ]:
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [ ]:
for col in df.columns:
    print(df[col].value_counts())

total_bill
13.42    3
21.01    2
20.69    2
10.33    2
10.34    2
        ..
29.03    1
27.18    1
22.67    1
17.82    1
18.78    1
Name: count, Length: 229, dtype: int64
tip
2.00    33
3.00    23
4.00    12
2.50    10
5.00    10
        ..
1.47     1
1.17     1
4.67     1
5.92     1
1.75     1
Name: count, Length: 123, dtype: int64
sex
Male      157
Female     87
Name: count, dtype: int64
smoker
No     151
Yes     93
Name: count, dtype: int64
day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64
time
Dinner    176
Lunch      68
Name: count, dtype: int64
size
2    156
3     38
4     37
5      5
1      4
6      4
Name: count, dtype: int64


In [ ]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [ ]:
x=df[ ['tip', 'sex', 'smoker', 'day', 'time', 'size']]
y=df[['total_bill']]

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
x_train

#feature encoding

from sklearn.preprocessing import LabelEncoder
le1=LabelEncoder()
le2=LabelEncoder()
le3=LabelEncoder()

x_train['sex']=le1.fit_transform(x_train['sex'])
x_train['smoker']=le2.fit_transform(x_train['smoker'])
x_train['time']=le3.fit_transform(x_train['time'])
print(x_train)

      tip  sex  smoker   day  time  size
228  2.72    1       0   Sat     0     2
208  2.03    1       1   Sat     0     2
96   4.00    1       1   Fri     0     2
167  4.50    1       0   Sun     0     4
84   2.03    1       0  Thur     1     2
..    ...  ...     ...   ...   ...   ...
106  4.06    1       1   Sat     0     2
14   3.02    0       0   Sun     0     2
92   1.00    0       1   Fri     0     2
179  3.55    1       1   Sun     0     2
102  2.50    0       1   Sat     0     3

[195 rows x 6 columns]


In [ ]:

x_test['sex']=le1.transform(x_test['sex'])
x_test['smoker']=le2.transform(x_test['smoker'])
x_test['time']=le3.transform(x_test['time'])
print(x_test)

      tip  sex  smoker   day  time  size
24   3.18    1       0   Sat     0     2
6    2.00    1       0   Sun     0     2
153  2.00    1       0   Sun     0     4
211  5.16    1       1   Sat     0     4
198  2.00    0       1  Thur     1     2
176  2.00    1       1   Sun     0     2
192  2.56    1       1  Thur     1     2
124  2.52    0       0  Thur     1     2
9    3.23    1       0   Sun     0     2
101  3.00    0       1   Fri     0     2
45   3.00    1       0   Sun     0     2
233  1.47    1       0   Sat     0     2
117  1.50    0       0  Thur     1     2
177  2.00    1       1   Sun     0     2
82   1.83    0       0  Thur     1     1
146  1.36    0       0  Thur     1     3
200  4.00    1       1  Thur     1     3
15   3.92    1       0   Sun     0     2
66   2.47    0       0   Sat     0     2
142  5.00    1       0  Thur     1     5
33   2.45    0       0   Sat     0     4
19   3.35    1       0   Sat     0     3
109  4.00    0       1   Sat     0     2
30   1.45    1  

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Apply OneHotEncoder to the 'day' column (index 3, as per current x_train/x_test structure)
# The remainder='passthrough' will keep the other columns ('tip', 'sex', 'smoker', 'time', 'size')
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(drop='first'), [3])], remainder='passthrough')

# Fit and transform x_train, storing the result in a new variable
x_train_encoded = ct.fit_transform(x_train)

# Transform x_test using the fitted ColumnTransformer, storing the result in a new variable
x_test_encoded = ct.transform(x_test)

In [ ]:
from sklearn.svm import SVR

svr=SVR()
y_pred = svr.fit(x_train_encoded, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
y_pred = svr.predict(x_test_encoded)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.5502250005312896

In [ ]:
from sklearn.model_selection import GridSearchCV
para_grid ={
    'kernel':['linear','poly','rbf','sigmoid'],
    'C':[1,5,10],
    'gamma':['scale','auto']
}
grid_search=GridSearchCV(estimator=svr,param_grid=para_grid,cv=5)
grid_search.fit(x_train_encoded,y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [1, 5, 10], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']})

In [ ]:
y_pred = grid_search.predict(x_test_encoded)

from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.6666814332339259